- 親手寫個檢索系統吧（ㄧ）文件預處理
- 親手寫個檢索系統吧（二）倒排索引
- 親手寫個檢索系統吧（三）索引壓縮

## 1.文件預處理

In [1]:
text = '''
 <body>
    <!-- JavaScript plugins (requires jQuery) -->
    <script src="http://code.jquery.com/jquery.js"></script>
    <!-- Include all compiled plugins (below), or include individual files as needed -->
    <script src="js/bootstrap.min.js"></script>

    <div class="container">
        <div class="page-header">
            <h3>About Me</h3>
        </div>
        <div class="page-info">

A web developer with experience in a variety of exciting projects, with the most up-to-date and relevant programming foundations available. My wide experience in
a diversity of technologies guides me with the best way to get your business success.
My interest in academic leads me to research in the field of NLP(Natural Language 
Processing). Other than the knowledge in CS/IT, I'm also a broad learner who loves 
to read each and every kind of books.
        </div>
    </div>
</body>
'''

In [3]:
import re

text = re.sub("<[^>]+>", "", text).strip()
text = text.split("\n\n")[1].replace("\n", " ")
print(text)

A web developer with experience in a variety of exciting projects, with the most up-to-date and relevant programming foundations available. My wide experience in a diversity of technologies guides me with the best way to get your business success. My interest in academic leads me to research in the field of NLP(Natural Language  Processing). Other than the knowledge in CS/IT, I'm also a broad learner who loves  to read each and every kind of books.


- （ㄧ）文件預處理 用NLTK提供的句子分割器

In [5]:
import nltk
nltk.download('punkt')
sent_segmenter = nltk.data.load('tokenizers/punkt/english.pickle')

sentences = sent_segmenter.tokenize(text)
print(sentences)

['A web developer with experience in a variety of exciting projects, with the most up-to-date and relevant programming foundations available.', 'My wide experience in a diversity of technologies guides me with the best way to get your business success.', 'My interest in academic leads me to research in the field of NLP(Natural Language  Processing).', "Other than the knowledge in CS/IT, I'm also a broad learner who loves  to read each and every kind of books."]


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cti110016\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
word_tokenizer = nltk.tokenize.regexp.WordPunctTokenizer()

tokenized_sentence = word_tokenizer.tokenize(sentences[0])
print(tokenized_sentence)
print(sentences[0].split(" "))

['A', 'web', 'developer', 'with', 'experience', 'in', 'a', 'variety', 'of', 'exciting', 'projects', ',', 'with', 'the', 'most', 'up', '-', 'to', '-', 'date', 'and', 'relevant', 'programming', 'foundations', 'available', '.']
['A', 'web', 'developer', 'with', 'experience', 'in', 'a', 'variety', 'of', 'exciting', 'projects,', 'with', 'the', 'most', 'up-to-date', 'and', 'relevant', 'programming', 'foundations', 'available.']


- （二）倒排索引

In [7]:
import requests
from pathlib import Path

fname = 'wsta_col_20k.gz'
my_file = Path(fname)
if not my_file.is_file():
    url = 'https://hyhu.me/resources/' + fname
    r = requests.get(url)

    # Save to the current directory
    with open(fname, 'wb') as f:
        f.write(r.content)

SSLError: HTTPSConnectionPool(host='hyhu.me', port=443): Max retries exceeded with url: /resources/wsta_col_20k.gz (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1129)')))

In [8]:
import gzip

raw_docs = []
with gzip.open(fname, 'rt') as f:
    for raw_doc in f:
        raw_docs.append(raw_doc)

print(len(raw_docs))
print(raw_docs[0])

FileNotFoundError: [Errno 2] No such file or directory: 'wsta_col_20k.gz'

In [9]:
from math import log, sqrt

# 給定一個查詢(String)和一個索引(Class)，回傳k個文件
def query_tfidf(query, index, k=10):

    # scores 儲存了docID和他們的TF-IDF分數
    scores = Counter()

    N = index.num_docs()

    for term in query:
        i = 0
        f_t = index.f_t(term)
        for docid in index.docids(term):
            # f_(d,t)
            f_d_t = index.freqs(term)[i]
            d = index.doc_len[docid]
            tfidf_cal = log(1+f_d_t) * log(N/f_t) / sqrt(d)
            scores[docid] += tfidf_cal
            i += 1

    return scores.most_common(k)

# 查詢語句
query = "south korea production"
# 預處理查詢，為了讓查詢跟索引內容相同
stemmed_query = nltk.stem.PorterStemmer().stem(query).split()
results = query_tfidf(stemmed_query, invindex)
for rank, res in enumerate(results):
    # e.g 排名 1 DOCID 176 SCORE 0.426 內容 South Korea rose 1% in February from a year earlier, the
    print("排名 {:2d} DOCID {:8d} SCORE {:.3f} 內容 {:}".format(rank+1,res[0],res[1],raw_docs[res[0]][:75]))

NameError: name 'invindex' is not defined

In [10]:
import nltk
from nltk.corpus import brown
from nltk.corpus import wordnet
#nltk.download('brown')
#nltk.download('wordnet')

# filtered_gold_standard 儲存移除稀有自後的文字組合與黃金標籤
filtered_gold_standard = {}

from collections import Counter
doc_freqs = Counter()

# lemmatize
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma

# 移除有稀有字的文字組合
def remove_word_pair(dictionary, li):
    for key in list(dictionary.keys()):
        if key[0] not in li or key[1] not in li:
            dictionary.pop(key, None)

# 讀取資料，同時將文字組合與相似度存進filtered_gold_standard
# 也將每一個獨特自行存到doc_freqs Counter
with open("set1.tab") as f:
    next(f)
    for line in f:
        line_list = line.split("\t")
        filtered_gold_standard[(line_list[0], line_list[1])] = float('%.2f' % float(line_list[2]))
        for i in range(2):
            doc_freqs[line_list[i]] = 0

bwn_par = brown.paras() # 未處理的Brown Corpus段落
norm_par = [] # 預處理過的Brown Corpus段落

# 進行預處理
for i in range(len(bwn_par)):
    para_sentence = []
    for j in range(len(bwn_par[i])):
        para_sentence.append([lemmatize(word.lower()) for word in bwn_par[i][j] if word.isalpha()])
    norm_par.append(sum(para_sentence,[]))

# 計算每個處理過的字的document frequency
for word in doc_freqs:
    for i in range(len(norm_par)):
        if word in norm_par[i]:
            doc_freqs[word] += 1
            
# 將document frequency < 8 的稀有字去除
for key in list(doc_freqs.keys()):
    if doc_freqs[key] < 8:
        doc_freqs.pop(key,None)

remove_word_pair(filtered_gold_standard, doc_freqs)

print(len(filtered_gold_standard))
print(filtered_gold_standard)

FileNotFoundError: [Errno 2] No such file or directory: 'set1.tab'